In [1]:
%load_ext autoreload
%autoreload 2

In [19]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.options.display.max_info_columns=500
pd.options.display.max_info_rows=550
# from tensorflow.keras.losses import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
# from sklearn.compose import ColumnTransformer
from sklearn import set_config; set_config(display='diagram')
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, cross_val_score, GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_percentage_error

In [3]:
from carbon_forecast.utils.preproc import load_n_preproc_all_avail
df = load_n_preproc_all_avail()
X = df.drop(columns=["company_name", "TCUID", "year", "intensity_1and2","intensity_scope3"])
y = df["intensity_1and2"]

Reading data...
Preproc on power % revenue
Preproc on Sector Revenue
Selecting the year 2013 required...
Dropping rows that are showing NaN on...
-> intensity_scope1CY2013
-> intensity_scope2CY2013
-> intensity_scope3CY2013
-> Revenue2013
Filling employee numbers with mean value... (subject to change later)
Filling EV with mean value... (subject to change later)
Calculating Employees & EV per Revenue...
Filling null Climate Strategy Score with 0...
Cleaning P/E Ratio...
Adding Scope 1 and Scope 2 data together...
Doing some column renaming...
Filling in Disclosure...
Dataset preproc-ed for the year 2013!👍
Reading data...
Preproc on power % revenue
Preproc on Sector Revenue
Selecting the year 2014 required...
Dropping rows that are showing NaN on...
-> intensity_scope1CY2014
-> intensity_scope2CY2014
-> intensity_scope3CY2014
-> Revenue2014
Filling employee numbers with mean value... (subject to change later)
Filling EV with mean value... (subject to change later)
Calculating Employees 

In [11]:
X.columns.tolist()

['Sector',
 'Employees / Revenue',
 'EV / Revenue',
 'climate_strategy_score',
 'disclosure',
 'pe_rat',
 'revenue',
 'nuclear_percentage_revenue',
 'thermal_coal_percentage_revenue',
 'ultra_deep_sea_drilling_percentage_revenue',
 'shale_oil_gas_percentage_revenue',
 'coal_percentage_revenue',
 'arctic_drilling_percentage_revenue',
 'oil_sands_percentage_revenue',
 'Abrasive product manufacturing',
 'Accounting, tax preparation, bookkeeping, and payroll services',
 'Adhesive manufacturing',
 'Advertising and related services',
 'Air and gas compressor manufacturing',
 'Air conditioning, refrigeration, and warm air heating equipment manufacturing',
 'Air purification and ventilation equipment manufacturing',
 'Air transportation',
 'Aircraft engine and engine parts manufacturing',
 'Aircraft manufacturing',
 'Alkalies and chlorine manufacturing',
 'All other basic inorganic chemical manufacturing',
 'All other chemical product and preparation manufacturing',
 'All other converted paper

In [7]:
from carbon_forecast.utils.tx import our_tx, tt_split
transformer = our_tx(X)
X_train_transformed, X_test_transformed, y_train, y_test = tt_split(X,y,transformer)
X_train_transformed

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [16]:
from carbon_forecast.utils.pca import our_pca
X_proj, X_test_proj = our_pca(X_train_transformed, X_test_transformed, 12)
X_proj

array([[ 5.90597312e-01, -5.08552911e-01,  9.89725990e-01, ...,
        -2.35129745e-02,  5.83725219e-02, -2.85710509e-01],
       [ 4.16288030e-01, -2.56176004e-01, -1.81792620e-01, ...,
        -7.16550366e-03,  8.00910377e-02, -2.64518123e-01],
       [-3.82123709e-01, -5.57040130e-02, -3.41239341e-02, ...,
        -2.34985576e-02,  8.13830635e-04, -8.75116333e-03],
       ...,
       [ 7.43391282e-01,  7.62657117e-01,  4.44439361e-02, ...,
        -1.70342221e-03,  9.20166094e-02, -2.69407801e-01],
       [-3.96119675e-01, -7.17362855e-02, -4.49078808e-02, ...,
        -7.59187938e-02, -3.12225205e-02, -4.17183723e-02],
       [-3.41484490e-01, -3.80065550e-01,  1.06404108e+00, ...,
        -3.22228584e-02, -3.72293869e-02, -5.92075666e-04]])

In [1]:
X_proj.shape

NameError: name 'X_proj' is not defined

In [13]:
model = XGBRegressor(n_estimators=1000, max_depth=10, eta=0.1, gamma=1, objective="reg:squarederror", eval_metric="mape")
cross_val_score(model, X_proj, y_train, cv=5, scoring="neg_mean_absolute_percentage_error")

array([-4.41150112, -2.19145205, -3.11015706, -7.54811267, -2.3330101 ])

In [17]:
model.fit(X_proj, y_train)


array([ 38.649742, 237.28459 ,   9.992757, ...,  88.206696, 186.98389 ,
        82.06866 ], dtype=float32)

In [21]:
y_pred = model.predict(X_test_proj)


In [22]:
mean_absolute_percentage_error(y_test, y_pred)

3.5448063267865053

In [23]:
y_pred -y_test

2011       1.998351
8276      81.508610
3526       2.189757
1598      83.462842
2776     178.428429
            ...    
15535      0.035578
11292     -6.014948
8121      -1.619497
5099     -32.156743
4806     -69.807343
Name: intensity_1and2, Length: 25530, dtype: float64